In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#copyright @lakshmi_vijayan

Mounted at /content/drive


In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import cv2
from keras.utils import img_to_array, load_img

# from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical ,Sequence
from keras import backend as K
from keras.layers import Input, Conv2D,AveragePooling2D,Add, MaxPooling2D, concatenate, Conv2DTranspose, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adadelta, Nadam ,Adam
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, TensorBoard

import os
# print(os.listdir(r'/content/drive/MyDrive/3. The cropped aerial image tiles and raster labels'))
# print(os.listdir(r'/content/drive/MyDrive/512_patches/'))
from glob import glob
from pathlib import Path
import shutil
from tqdm import tqdm_notebook
from random import sample, choice

# Any results you write to the current directory are saved as output.

In [ ]:
import tensorflow as tf
from keras import callbacks, optimizers
from keras.models import Model, load_model, model_from_json
from keras.preprocessing import image
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, BatchNormalization, Conv2DTranspose, Concatenate, UpSampling2D, Conv1DTranspose, DepthwiseConv2D
from keras.layers import Dropout
from keras.activations import gelu
from keras.callbacks import ModelCheckpoint
from keras import backend as K
 

import matplotlib.pyplot as plt
%matplotlib inline
 
K.set_image_data_format('channels_last')
import time


## EDA

In [ ]:
dataset_path = Path(r"/content/drive/MyDrive/MAS_dataset_prep/new_mas_patch")
list(dataset_path.iterdir())

[PosixPath('/content/drive/MyDrive/MAS_dataset_prep/new_mas_patch/test_labels'),
 PosixPath('/content/drive/MyDrive/MAS_dataset_prep/new_mas_patch/val_labels'),
 PosixPath('/content/drive/MyDrive/MAS_dataset_prep/new_mas_patch/train'),
 PosixPath('/content/drive/MyDrive/MAS_dataset_prep/new_mas_patch/val'),
 PosixPath('/content/drive/MyDrive/MAS_dataset_prep/new_mas_patch/test'),
 PosixPath('/content/drive/MyDrive/MAS_dataset_prep/new_mas_patch/train_labels')]

In [ ]:
def tree(directory):
    print(f'+ {directory}')
    for path in sorted(directory.rglob('*')):
        depth = len(path.relative_to(directory).parts)
        spacer = '    ' * depth
        print(f'{spacer}+ {path.name}')       
#tree(dataset_path)

In [ ]:
dataset_path

In [ ]:
train_imgs = sorted(list((dataset_path/"train").glob("*.png")))
train_labels = sorted(list((dataset_path/"train_labels").glob("*.png")))

val_imgs = sorted(list((dataset_path/"val").glob("*.png")))
val_labels = sorted(list((dataset_path/"val_labels").glob("*.png")))

test_imgs = sorted(list((dataset_path/"test").glob("*.png")))
test_labels = sorted(list((dataset_path/"test_labels").glob("*.png")))

(len(train_imgs),len(train_labels)), (len(test_imgs),len(test_labels))

img_size = 512

In [ ]:
test_imgs[4]

In [ ]:
test_labels[4]

In [ ]:
len(train_imgs)

In [ ]:
def make_pair(img,label,dataset):
    pairs = []
    for im in img:
        pairs.append((im , dataset / label / (im.stem +".png")))
    
    return pairs

In [ ]:
train_pair = make_pair(train_imgs,"train_labels", dataset_path)
val_pair = make_pair(val_imgs,"val_labels", dataset_path)
test_pair = make_pair(test_imgs, "test_labels", dataset_path)

In [ ]:
temp = choice(val_pair)
temp


(PosixPath('/content/drive/MyDrive/MAS_dataset_prep/new_mas_patch/val/22978945_15_0.pngpatch_01.png'),
 PosixPath('/content/drive/MyDrive/MAS_dataset_prep/new_mas_patch/val_labels/22978945_15_0.pngpatch_01.png'))

In [ ]:
temp = choice(train_pair)
img = img_to_array(load_img(temp[0], target_size=(img_size,img_size)))
mask = img_to_array(load_img(temp[1], target_size = (img_size,img_size)))
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(img/255)
plt.subplot(122)
plt.imshow(mask/255, cmap='gray')

In [ ]:
class_map_df = pd.read_csv(r"/content/drive/MyDrive/test/label_class_dict (2).csv")
class_map_df


,name,r,g,b
0,background,0,0,0
1,building,255,255,255


In [ ]:
class_map_df['r'][1]

255

In [ ]:
class_map_df

,name,r,g,b
0,background,0,0,0
1,building,255,255,255


In [ ]:
class_map = []
for index,item in class_map_df.iterrows():
    class_map.append(np.array([item['r'], item['g'], item['b']]))
    
print(class_map)
print(len(class_map))

[array([0, 0, 0]), array([255, 255, 255])]
2


In [ ]:
def assert_map_range(mask,class_map):
    mask = mask.astype("uint8")
    for j in range(img_size):
        for k in range(img_size):
            assert mask[j][k] in class_map , tuple(mask[j][k])

In [ ]:
def form_2D_label(mask,class_map):
    mask = mask.astype("uint8")
    label = np.zeros(mask.shape[:2],dtype= np.uint8)
    
    for i, rgb in enumerate(class_map):
        label[(mask == rgb).all(axis=2)] = i
    
    return label

In [ ]:
lab = form_2D_label(mask,class_map)
np.unique(lab,return_counts=True)

(array([0, 1], dtype=uint8), array([192378,  69766]))

In [ ]:

class DataGenerator(Sequence):
    'Generates data for Keras'
    
    def __init__(self, pair, class_map, batch_size=4, dim=(512,512,3), shuffle=True):
        'Initialization'
        self.dim = dim
        self.pair = pair
        self.class_map = class_map
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.pair) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [k for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.pair))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        batch_imgs = list()
        batch_labels = list()

        # Generate data
        for i in list_IDs_temp:
            # Store sample
            img = load_img(self.pair[i][0] ,target_size=self.dim)
            img = img_to_array(img)/255.
            batch_imgs.append(img)

            label = load_img(self.pair[i][1],target_size=self.dim)
            label = img_to_array(label)
            label = form_2D_label(label,self.class_map)
            label = to_categorical(label , num_classes = 2) #2
            batch_labels.append(label)
            
        return np.array(batch_imgs) ,np.array(batch_labels)

In [ ]:
img_size = 512

In [ ]:
train_generator = DataGenerator(train_pair,class_map,batch_size=2, dim=(img_size,img_size,3) ,shuffle=True)
train_steps = train_generator.__len__()
train_steps

616

In [ ]:
# X,y = train_generator.__getitem__(1)
# y.shape

In [ ]:
# X_train = []
# y_train = []
# for i in range(train_steps):
#     X,y = train_generator.__getitem__(i)
#     X_train.append(X)
#     y_train.append(y)

In [ ]:
val_generator = DataGenerator(val_pair, class_map, batch_size=2, dim=(img_size,img_size,3) ,shuffle=True)
val_steps = val_generator.__len__()
val_steps

18

In [ ]:
test_generator = DataGenerator(test_pair, class_map, batch_size=4, dim=(img_size,img_size,3) ,shuffle=True)
test_steps = test_generator.__len__()
test_steps

22

In [ ]:
X_test = []
y_test = []
for i in range(test_steps):
    X,y = test_generator.__getitem__(i)
    X_test.append(X)
    y_test.append(y)

In [ ]:
image_shape = (512, 512, 3)
from keras import layers

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3]) #,3
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

# def dice_coef(y_true, y_pred):

#     y_true_f = K.flatten(y_true)
#     y_pred_f = K.flatten(y_pred)
#     intersection = K.sum(y_true_f * y_pred_f)
#     return (2. * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0) 
 
def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [ ]:
def atrous_1(input_tensor, num_filters, dilation):
  b0=Conv2D(num_filters,(1,1), padding='same',dilation_rate=dilation[1], use_bias=True)(input_tensor)
  b0=BatchNormalization()(b0)
  # b0 = Dropout(0.3)(b0)
  b0=Activation("relu")(b0)

  return b0

In [ ]:
def atrous(input_tensor, num_filters, dilation):

  b1=Conv2D(num_filters,kernel_size = (3,1),padding='same',dilation_rate=dilation[0], use_bias=True)(input_tensor)  
  b1=BatchNormalization()(b1)
  b1=Activation("relu")(b1)

  b1=Conv2D(num_filters,kernel_size = (1,3),padding='same',dilation_rate=dilation[1], use_bias=True)(b1)
  b1=BatchNormalization()(b1)
  b1_cas=Activation("relu")(b1)

  x=Concatenate()([input_tensor, b1_cas])
  b1_cas=Activation("relu")(x)

  return b1_cas

In [ ]:
def conv1x1(input_tensor, num_filters):
  b1 = Conv2D(filters = num_filters,kernel_size = (1,1),padding="same", use_bias=True)(input_tensor)
  return b1

In [ ]:
def center_block(input_tensor, num_filters):
  dilation_rates = [ [18,18], [12,12], [4,4], [1,1]]
  # a0 = conv1x1(input_tensor, num_filters)
  # a0 = cont_drop(a0)

  a1 = atrous_1(input_tensor, num_filters, dilation_rates[0])
  a2 = atrous(input_tensor, num_filters, dilation_rates[1])
  a3 = atrous(input_tensor, num_filters, dilation_rates[2])
  a4 = atrous(input_tensor, num_filters, dilation_rates[3])

  a1 = conv1x1(a1, num_filters)
  a2 = conv1x1(a2, num_filters)
  a3 = conv1x1(a3, num_filters)
  a4 = conv1x1(a4, num_filters)

  a1 = conv1x1(a1, num_filters)
  a2 = conv1x1(a2, num_filters)
  a3 = conv1x1(a3, num_filters)
  a4 = conv1x1(a4, num_filters)


  # out_shape=int(input_shape[0]/16)
  # a5 = GlobalAveragePooling2D(out_shape,out_shape)(input_tensor)
  # a5 = UpSampling2D((out_shape,out_shape))(a5)

  x=Concatenate()([a1,a2,a3,a4])


  return x

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage.util import random_noise
from skimage import feature
from PIL import Image
from numpy import asarray

In [ ]:

from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, Input
from keras.layers import Activation, Concatenate, Conv2D, Multiply

def channel_attention_module(x, ratio=8):
    batch, _, _, channel = x.shape

    ## Shared layers
    l1 = Dense(channel//ratio, activation="relu", use_bias=False)
    l2 = Dense(channel, use_bias=False)

    ## Global Average Pooling
    x1 = GlobalAveragePooling2D()(x)
    x1 = l1(x1)
    x1 = l2(x1)

    ## Global Max Pooling
    x2 = GlobalMaxPooling2D()(x)
    x2 = l1(x2)
    x2 = l2(x2)

    ## Add both the features and pass through sigmoid
    feats = x1 + x2
    feats = Activation("sigmoid")(feats)
    feats = Multiply()([x, feats])

    return feats

In [ ]:
def spatial_attention_module(x):

    ## Average Pooling
    x1 = tf.reduce_mean(x, axis=-1)  #from 128 give 1 channel
    x1 = tf.expand_dims(x1, axis=-1)  #-1 -> last channel

    ## Max Pooling
    x2 = tf.reduce_max(x, axis=-1)
    x2 = tf.expand_dims(x2, axis=-1)
    

    ## Concatenat both the features
    feat = Concatenate()([x1, x2])
    ## Conv layer
    feats1 = Conv2D(1, kernel_size=3, padding="same", activation="sigmoid", dilation_rate=1)(feat)
    feats2 = Conv2D(1, kernel_size=3, padding="same", activation="sigmoid", dilation_rate=4)(feat)
    feats3 = Conv2D(1, kernel_size=3, padding="same", activation="sigmoid", dilation_rate=8)(feat)
    feats4 = Conv2D(1, kernel_size=3, padding="same", activation="sigmoid", dilation_rate=12)(feat)

    feats = Multiply()([x, feats1, feats2, feats3, feats4])

    return feats

In [ ]:
def cbam(x):
    x = channel_attention_module(x)
    x = spatial_attention_module(x)
    return x


In [ ]:
# Building Unet by dividing encoder and decoder into blocks

from keras.models import Model
from keras.layers import Input, Conv2D,Dot, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate


def conv_block(input, num_filters):
    x = Conv2D(num_filters, (3,3), padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network. 
    x = Activation("relu")(x)

    x = Conv2D(num_filters, (3,1), padding="same")(x)      #1  
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)

    x = Conv2D(num_filters, (1,3), padding="same")(x)     #1
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)

    return x


def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p   


def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):

    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    ASPP = center_block(p4, 1024)

    s4 = cbam(s4)
    d1 = decoder_block(ASPP, s4, 512)
    s3 = cbam(s3)
    d2 = decoder_block(d1, s3, 256)

    r_d1 = DepthwiseConv2D((5,5),strides=1, padding="same")(d1)
    r_d1 = Conv2DTranspose(512, (2,2),strides=2, padding="same")(r_d1)

    fusion_feat12 = Concatenate()([r_d1, d2])
    s2 = cbam(s2)
    d3 = decoder_block(fusion_feat12, s2, 128)
    s1 = cbam(s1)
    d4 = decoder_block(d3, s1, 64)
  

    outputs = Conv2D(2, 1, padding="same", activation="sigmoid")(d4)  #Binary
#boundary refining
    mod = atrous(outputs, 64, [8,12])
    output = Conv2D(2, 1, padding="same", activation="sigmoid")(mod)  #Binary
    x = Add()([outputs, output])
    outputs = Conv2D(2, 1, padding="same", activation="sigmoid")(x)  #Binary


    model = Model(inputs, outputs, name="Modified_Attention_Unet_with_Asymmetrical_feature_fusion_unit")
    return model


In [ ]:

smooth=1.

def dice_coef(y_true, y_pred):
    y_true = K.flatten(y_true)
    

    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred)
    return (2.0 * intersection + smooth) / (union + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    return dice_coef_loss(y_true, y_pred) + bce(y_true, y_pred)

def iou(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true + y_pred)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac
def iou_loss(y_true, y_pred):
  return 1 - iou(y_true, y_pred)


In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
input_shape = (image_shape[0], image_shape[1], image_shape[2])
model = build_unet(input_shape)
model.compile(optimizer=Adam(lr=1e-3), loss=[dice_coef_loss], metrics=[iou, dice_coef,'acc', recall_m, precision_m, f1_m])

In [ ]:
model.summary()

In [ ]:
model.optimizer.get_config()

Model architecture Analysis

In [ ]:
layers = model.layers

In [ ]:
print("No. of layers for our model = ",len(layers))

In [ ]:
filters, biases = model.layers[1].get_weights()
print(layers[1].name, filters.shape)  #filter details

In [ ]:
fig1=plt.figure(figsize=(8, 12))
columns = 8
rows = 8
n_filters = columns * rows
for i in range(1, n_filters +1):
    f = filters[:, :, :, i-1]
    fig1 =plt.subplot(rows, columns, i)
    fig1.set_xticks([])  #Turn off axis
    fig1.set_yticks([])
    plt.imshow(f[:, :, 0]) #Show only the filters from 0th channel (R)
    #ix += 1
plt.show()    

In [ ]:
#Define a new truncated model to only include the conv layers of interest
conv_layer_index = [1, 15, 24]  #TO define a shorter model
outputs = [model.layers[i].output for i in conv_layer_index]
model_short = Model(inputs=model.inputs, outputs=outputs)
model_short.summary()

In [ ]:
img_path='/content/drive/MyDrive/dataset/3. The cropped image tiles and raster labels/test/image/103.tif' 

#Load the input image
img = load_img(img_path, target_size=(512, 512))  
# Convert ht image to Array of dimension (150,150,3)
x   = img_to_array(img)                           
x   = x.reshape((1,) + x.shape)                   
# Rescale by 1/255
x /= 255.0

In [ ]:
def mean_iou(y_pred, y_true):
    if y_pred.shape.ndims > 1:
        y_pred = array_ops.reshape(y_pred, [-1])

    if y_true.shape.ndims > 1:
        y_true = array_ops.reshape(y_true, [-1])
    intersection = K.sum(K.sum(K.abs(y_true * y_pred), axis=-1))
    union = K.sum(K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1))

    return intersection / union

Model training

In [ ]:
# mc = ModelCheckpoint(mode='max', filepath=checkpoint_path, monitor='val_iou', save_weights_only='True', verbose=1)
es = EarlyStopping(mode='max', monitor='val_iou', patience=5, verbose=0)
# tb = TensorBoard(log_dir="logswhu/", histogram_freq=0, write_graph=True, write_images=False)
# rl = ReduceLROnPlateau(monitor='val_acc',factor=0.1,patience=5,verbose=1,mode="max",min_lr=0.0001)
# cv = CSVLogger("/content/drive/MyDrive/logwhu.csv" , append=True , separator=',')

In [ ]:
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

/usr/local/lib/python3.8/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [ ]:
start = time.time()

results = model.fit_generator(train_generator , steps_per_epoch=train_steps ,epochs=8,
                              validation_data=val_generator,validation_steps=val_steps,callbacks=[es])
stop = time.time()
print(f"Training time: {stop - start}s")
# from tensorflow import keras
# model = keras.models.load_model('/content/drive/MyDrive/new_modelmas.h5')

In [ ]:
img_mask = choice(test_pair)
img= img_to_array(load_img(img_mask[0] , target_size= (img_size,img_size)))
gt_img = img_to_array(load_img(img_mask[1] , target_size= (img_size,img_size)))

In [ ]:
def make_prediction(model,img_path,shape):
    img= img_to_array(load_img(img_path , target_size= shape))/255.
    img = np.expand_dims(img,axis=0)
    labels = model.predict(img)
    labels = np.argmax(labels[0],axis=2)
    return labels

In [ ]:
pred_label = make_prediction(model, img_mask[0], (img_size,img_size,3))
pred_label.shape

1/1 [==============================] - 5s 5s/step


(512, 512)

In [ ]:

def form_colormap(prediction,mapping):
    h,w = prediction.shape
    color_label = np.zeros((h,w,3),dtype=np.uint8)    
    color_label = mapping[prediction]
    color_label = color_label.astype(np.uint8)
    return color_label


In [ ]:
pred_colored = form_colormap(pred_label,np.array(class_map))

In [ ]:
X_test = []
y_test = []
for i in range(test_steps):
    X,y = test_generator.__getitem__(i)
    X_test.append(X)
    y_test.append(y)

In [ ]:
#
count = 0
for i in test_pair:
  count +=1
  if count <= 30:
    img_mask = i
    img= img_to_array(load_img(img_mask[0] , target_size= (img_size,img_size)))
    gt_img = img_to_array(load_img(img_mask[1] , target_size= (img_size,img_size)))
    pred_label = make_prediction(model, img_mask[0], (img_size,img_size,3))
    pred_colored = form_colormap(pred_label,np.array(class_map))
    plt.figure(figsize=(30,30))
    plt.subplot(131);plt.title('Original Image')
    plt.imshow(img/255.)
    plt.subplot(132);plt.title('True labels')
    plt.imshow(gt_img/255.)
    plt.subplot(133)
    plt.imshow(pred_colored/255.);plt.title('predicted labels')
  


In [ ]:
testiou_list = []
testmiou_list = []
testdice_list = []
OA = []
precision = []
recall = []
f1 = []
dice = []
start = time.time()

for i in range(test_steps):
  test_acc = model.evaluate(X_test[i], y_test[i])
  testiou_list.append(test_acc[1])
  # testmiou_list.append(test_acc[2])
  testdice_list.append(test_acc[2])
  # OA.append(test_acc[4])
  f1.append(test_acc[5])
  precision.append(test_acc[4])
  recall.append(test_acc[3])

  stop = time.time()
print(f"Testing time: {stop - start}s")


In [ ]:
avg_test_iou = sum(testiou_list) / len(testiou_list)
# avg_test_miou = sum(testmiou_list) / len(testmiou_list)

avg_test_dice = sum(testdice_list) / len(testdice_list)
# OA = sum(OA) / len(OA)
f1 = sum(f1) / len(f1)
precision = sum(precision) / len(precision)
recall = sum(recall) / len(recall)

print("Test IoU: ", avg_test_iou)
# print("Test mIoU: ", avg_test_miou)
print("Test dice coeff: ",avg_test_dice)
print("Test f1: ",f1)
print("Test precision: ",precision)
print("Test recall: ",recall)

Test IoU:  0.8462419455701654
Test dice coeff:  0.9164391783150759
Test f1:  0.9141583659432151
Test precision:  0.9209287816827948
Test recall:  0.9172993139787153


# END